# Keras tutorial

Keras - это высокоуровневый API, написанный на Python и способный работать поверх TensorFlow, CNTK или Theano.

__Преимущество:__ Переход от идеи к результату с наименьшими затратами по времени

Вам скорее всего подойдет Keras, если от библиотеки DL вам нужно:

- Легко и быстро создавать прототипы (удобство, модульность и расширяемость)
- Поддерживать как сверточные сети, так и рекуррентные сети, а также их комбинации
- Работать без проблем на CPU или GPU

Удобство для пользователя: Keras - это API, разработанный для людей, а не для машин

## Запускаем Keras за 30 секунд:

In [191]:
from keras.models import Sequential
import keras
model = Sequential()

Стакаем слои через .add():

In [192]:
from keras.layers import Dense

model.add(Dense(units=64, activation='relu', input_dim=100)) # input_dim=100 <=> input_shape=(100, )
model.add(Dense(units=10, activation='softmax'))

In [193]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_61 (Dense)             (None, 64)                6464      
_________________________________________________________________
dense_62 (Dense)             (None, 10)                650       
Total params: 7,114
Trainable params: 7,114
Non-trainable params: 0
_________________________________________________________________


Для модели необохимо опредеить оптимизируемый лосс, оптимизатор и собираемые метрики качества

In [194]:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Или альтернативный вариант:

In [195]:
from keras import metrics

model.compile(loss=metrics.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True),
             metrics=[metrics.categorical_accuracy])

Теперь можем запустить процесс обучения, итурируясь по батчам:

In [196]:
# Сначала сгенерируем случайные обучающие данные:
import numpy as np
from keras.utils import np_utils

x_train = np.random.random((1000, 100))
y_train = np_utils.to_categorical(np.random.randint(10, size=(1000, 1)))

In [197]:
# x_train and y_train are Numpy arrays --just like in the Scikit-Learn API.
model.fit(x_train, y_train, epochs=100, batch_size=32, verbose=1)

Epoch 1/100
1000/1000 [==============================] - 5s 5ms/step - loss: 2.3492 - categorical_accuracy: 0.1080
Epoch 2/100
1000/1000 [==============================] - 0s 179us/step - loss: 2.3027 - categorical_accuracy: 0.1270

__Вопрос__: почему модель обучается?

Или можно запустить однократно обучение на одном конкретном батче:

In [198]:
x_batch = x_train[:10]
y_batch = y_train[:10]
model.train_on_batch(x_batch, y_batch)
# возвращает скаляр - лосс на обучающем батче

[0.31208184, 1.0]

Оценим модель, это можно сделать в одну строчку.

__Вопрос__: угадайте какой accuracy даст модель на валидационной выборке? 

In [199]:
test_size = 100
x_test = np.random.random((test_size, 100))
y_test = np_utils.to_categorical(np.random.randint(10, size=(test_size, 1)))

loss_and_metrics = model.evaluate(x_test, y_test, batch_size=128)
loss_and_metrics

100/100 [==============================] - 2s 17ms/step


[4.0238857269287109, 0.11999999731779099]

Или можно сделать предсказание на новом наборе данных:

In [200]:
classes = model.predict(x_test, batch_size=128)
classes

array([[  6.99881837e-02,   1.31185551e-03,   2.98728875e-04,
          1.45911928e-02,   3.34465206e-02,   4.26112078e-02,
          1.91896688e-04,   7.95170903e-01,   7.09834695e-03,
          3.52911316e-02],
       [  1.14395199e-02,   3.43768626e-01,   3.60790700e-01,
          1.20582394e-02,   7.20033422e-03,   9.66453701e-02,
          6.49059191e-02,   9.45590287e-02,   2.20208225e-04,
          8.41210689e-03],
       [  5.35921574e-01,   1.35980244e-03,   1.37950301e-01,
          1.65477544e-02,   1.65208839e-02,   2.02700915e-03,
          1.70590961e-03,   2.25587517e-01,   7.70392735e-03,
          5.46753891e-02],
       [  7.45125711e-01,   1.06682397e-01,   7.36264586e-02,
          2.24206917e-04,   2.93229544e-03,   1.11382804e-03,
          1.00577669e-02,   4.51383367e-03,   5.39012253e-02,
          1.82230095e-03],
       [  3.52082513e-02,   2.46005096e-02,   3.84414122e-02,
          4.07874256e-01,   8.99129733e-03,   3.13102186e-01,
          4.07098734e-04

__Итого:__
1. Получить данные: x_train, y_train, x_test, y_test
2. Объявить модель: слои, их параметры и тд.
3. Скомпилировать модель: лосс, оптимизатор, метрики
4. fit: обучение
5. evaluate: оцениваем качество модели
6. predict: предсказываем на отложенной выборке

## Sequential():

Два способа использования Squential.
Первый:

In [203]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential([
    Dense(32, input_shape=(784,10)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])

Второй, с использованием add():

In [204]:
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))

Посмотрим внимательнее на объявление модели. 

Модели требуется указать размерность входных данных. Для этого в первый слой для Sequential (и только первый слой, т.к все последующие слои автоматически считают размернсоти) требуется передать эту информацию. Это можно сделать двумя способами: 
- Передать в первый слов параметр input_dim (только 1-мерная размерность данных) или input_shape (n мерная размерность). input_shape - tuple целочисленных значений, размерность батча не указывается
- В качестве первого слоя передать в Sequential следующий слой: ```Input(shape=(...))```

## Compilation

Перед обучение модели требуется сконфигурировать обучающий процесс. Для этого в метод ```model.complie()``` требуется передать три аргумента: 
- Optimizer. Это может быть как string, обозначающий один из оптимизаторов, так и instance одного из оптимизаторов
https://keras.io/optimizers/

- Loss. Функция, которую будет оптимизировать модель в ходе обучения. Это может быть как string (например, 'categorical_crossentropy' или 'mse'), так и instance класса Losses https://keras.io/losses/
- Metrics. Список метрик, которые будет собирать модель в ходе обучения. В список можно добавлять как string (напрмиер, ```metrics=['accuracy']```), так и метрики из класса Metrics https://keras.io/metrics/. Также можно добавлять кастомные метрики

Все примеры можно найти ниже:

In [205]:
# Многклассовая классификация
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Бинарная классификация
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Для задачи регрессии
model.compile(optimizer='rmsprop',
              loss='mse')

# Если хотите сделать кастомную функцию:
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])

## Training:

На обучение модели подаются данные для обучения: ```x_train``` и ```y_train```. Оба объекта - numpy массивы. Для обучения модели используется метод ```model.fit(...) ```
В fit подаются следующие аргументы:
- x: обучающая выборка
- y: ответы для обучающей выборки
- batch_size: размерность батча
- epochs: число эпох
- verbose: Integer. 0, 1, или 2. Режим оповещений в ходе обучения. 0 = тихий режим, 1 = progress bar, 2 = одна запись на одну эпоху

и т.д, см подробнее: https://keras.io/models/model/

In [206]:
# Пример: Задача бинарной классификации

# Как и описали выше, сначала объявляем Sequential() модель и накидываем слоев:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid')) # на выходе одно число, так как задача бинарной классификации

# Теперь делаем compile()
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Как обычно, сгенерируем рандомно данные
import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(2, size=(1000, 1))

# И, наконец, запустим обучение на данных. Данные подаются батчами по 32
model.fit(data, labels, epochs=3, batch_size=32, verbose=1)

Epoch 1/3
1000/1000 [==============================] - 4s 4ms/step - loss: 0.6995 - acc: 0.5320
Epoch 2/3
1000/1000 [==============================] - 0s 165us/step - loss: 0.6905 - acc: 0.5390
Epoch 3/3
1000/1000 [==============================] - 0s 183us/step - loss: 0.6832 - acc: 0.5520

Чтобы оценить качество модели, воспользуемся методом ```model.evaluate(...)```
Для этого метода подаются аргументы: 
- x: тествоая выборка
- y: ответы на тестовую выборку
- batch_size: размер батча

In [207]:
data_test = np.random.random((1000, 100))
labels_test = np.random.randint(2, size=(1000, 1))
model.evaluate(x=data_test, y=labels_test)

1000/1000 [==============================] - 1s 1ms/step


[0.70726329898834228, 0.51400000000000001]

__Вопрос__: в случае многоклассовой классификации какая функции активации будет использоваться в последнем слое?

In [210]:
# теперь многоклассовая задача:

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Генерируем данные:
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

# Ответы переводим в one-hot вектора
one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

# Train the model, iterating on the data in batches of 32 samples
model.fit(data, one_hot_labels, epochs=2, batch_size=32, verbose=1)

Epoch 1/2
1000/1000 [==============================] - 3s 3ms/step - loss: 2.3528 - acc: 0.1100
Epoch 2/2
1000/1000 [==============================] - 0s 108us/step - loss: 2.3190 - acc: 0.1260


__Вопрос:__ зачем мы применяем np_utils.to_categorical для задачи многоклассовой классификации?

Кросс-Энтропия (=logloss):
<img src="https://www.oreilly.com/library/view/machine-learning-with/9781787121515/assets/59dffc6c-561a-42a0-9a54-7a13b800ca7d.png" alt="Drawing" style="width: 300px;"/>

Снова оценим качество нашей модели:

In [211]:
data_test = np.random.random((1000, 100))
labels_test = np.random.randint(10, size=(1000, 1))
labels_test = keras.utils.to_categorical(labels_test, num_classes=10)
model.evaluate(x=data_test, y=labels_test)

1000/1000 [==============================] - 1s 1ms/step


[2.3552984504699706, 0.114]

## Ещё немного про Keras models

Модели Keras имеют следующий набор методов и аттрибутов:
- ```model.layers``` - список слоёв. К каждому элементу из списка можно обратиться
- ```model.inputs``` - тезор входных данных
- ```input_shape``` - размерность входных данных
- ```output_shape``` - размерность выходных данных
- ```model.outputs``` - тензор output
- ```model.summary()``` - печаатет информацию по вашей модели: слои, их параметры, число обучаемых параметров и т.д
- ```model.get_layer(layer_name)``` - возвращает слой по его имени
- ```model.get_config()``` - возвращает словарь: конфиг модели. Модель может быть из него собрана, пример будет ниже
- ```model.get_weights() ``` - возвращает список всех вессов модели, как  numpy массивы
- ```model.set_weights(weights)``` - устанавливает веса в модели

Ниже показаны примеры методов и аттрибутов, описанных выше:

In [212]:
model.layers

In [215]:
print(model.input_shape)
print(model.output_shape)

(None, 100)
(None, 10)


In [216]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_72 (Dense)             (None, 32)                3232      
_________________________________________________________________
dense_73 (Dense)             (None, 10)                330       
Total params: 3,562
Trainable params: 3,562
Non-trainable params: 0
_________________________________________________________________


##  Keras layers

Все слои в Keras имеют ряд общих методов и аттрибутов:
- ```layer.input``` - тензор входных данных
- ```layer.output``` - тензор выходных данных
- ```layer.input_shape``` - размерность входных данных
- ```layer.output_shape``` - размерность выходных данных

- ```layer.get_weights()``` - возвращает веса слоя
- ``` layer.set_weights(weights)``` - устанавливает веса слоя
- ```layer.get_config()``` - возвращает конфиг слоя

Ниже показаны примеры методов и аттрибутов, описанных выше:

In [219]:
# Возьмем кокой-нибудь слой:
model.layers

In [220]:
layer = model.layers[0]
print(layer.input_shape)
print(layer.output_shape)

(None, 100)
(None, 32)


In [221]:
layer.get_weights()

[array([[ 0.13789529,  0.00647529, -0.1463391 , ..., -0.16859421,
          0.11149354,  0.1042473 ],
        [ 0.10474449, -0.21291073,  0.21348844, ...,  0.14255466,
          0.19132593,  0.14750127],
        [-0.06924685, -0.13265613,  0.17940912, ...,  0.13551861,
          0.14650963,  0.1920817 ],
        ..., 
        [ 0.19784911, -0.18914668,  0.10658906, ..., -0.02052944,
         -0.13596351,  0.05952622],
        [ 0.19077006, -0.02003885,  0.19484691, ...,  0.04502568,
         -0.06472328, -0.09055799],
        [ 0.07095609,  0.0257611 , -0.01288384, ...,  0.04477685,
         -0.18244171, -0.08690173]], dtype=float32),
 array([-0.01320225, -0.00543841,  0.00027236, -0.01364938, -0.00962233,
        -0.00021641, -0.00340199, -0.00951376, -0.00626326, -0.00545387,
        -0.0103591 , -0.01655523,  0.00086164, -0.00398619, -0.00560309,
        -0.00135736, -0.00108287, -0.00290001,  0.00070663, -0.00648622,
        -0.00053374,  0.01009944, -0.00014616, -0.01234352, -0.00

### Далее поговорим про каждый тип слоя отдельно:

### InputLayer

Может пригодиться ровно один раз, для передачи информации нейронной сети о размерности входных данных

In [0]:
from keras.layers import InputLayer
model = Sequential()
model.add(InputLayer(input_shape=(1,)))
model.add(Dense(100))
model.summary()

### Dense:

Ниже представлены две картинки. На одной как обычно рисуют нейронные сети с линейными слоями, на второй - как реализовывать линейные слои через умножение матриц. Такая реализация позволяет эффективно подавать элементы батчами (т.е в одном этапе обучения участвуют сразу несколько элементов)

На данной картинке обучающая выборка состоит из 3 элементов, каждый из которых имеет одну фичу:

<img src="https://ml-cheatsheet.readthedocs.io/en/latest/_images/nn_with_matrices_displayed.png" alt="Drawing" style="width: 700px;"/>

In [222]:
# Пример
model = Sequential()
model.add(Dense(32, input_dim=16))
# Модель принимает на вход данные размерностью (*, 16), где * - размер батча
# На выходе модель выдает матрицу размерностью (*, 32), где * - размер батча

Также слой Dense имеет полезный параметр ```activation```, отвечающий за функцию активации. Она применяется поэлементно сразу после Dense слоя 

__Вопрос:__ Пусть i-ый слой имеет выходную размерность (n,m), а i+1-ый слой: Dense(100). Сколько обучаемых параметров у такого слоя?

### Flatten:

Разворачивает весь тензор в одномерный вектор. 
Например, матрица размера ```(n, m)``` превратится в вектор размера ``` n*m```

In [223]:
from keras.layers import Flatten
model = Sequential()
model.add(Flatten())

### Conv2D

<img src="https://hsto.org/files/16a/6ac/0d3/16a6ac0d3e1348db82d29d7ff8933ee8.jpg" alt="Drawing" style="width: 400px;"/>

<img src="https://www.cntk.ai/jup/cntk103d_conv2d_final.gif" alt="Drawing" style="width: 400px;"/>

<img src="https://i.stack.imgur.com/FjvuN.gif" alt="Drawing" style="width: 400px;"/>

Сonv2D - свертка, предназначенная только для тензоров размерности 3. Обычно размерность обозначают ```(a, b, num_channels)```. ```num_channels``` - канальность изображения. 
Kernel (окно) свертки имеет размерность ```(m, k, num_channels)```, где ```num_channels``` - число каналов (третья размерность) входного тензора. Она обязательно совпадает с ```num_channels``` входного изображения. 


Слой Conv2D в Keras имеет следующие параметры: 
- ```filters:int``` - число фильтров. Иными словами, сколько разных Kernel мы применим к нашей картинке. Каждый Kernel имеет размерность ```(m, k, num_channels)```, у каждого ```m*k*num_channels``` параметров обучаются

- ```kernel_size:int or tuple/list of 2 int``` - tuple из двух int. В наших обозначениях это (m, k)

- ```activation:str``` - функция активации, которая применяется сразу после применения слоя Conv2D
- ```strides: int or tuple/list of 2 int``` - отвечает за stride по вектрикали и горизонтали
- ```padding:str``` - принимает значение либо 'valid', либо 'same',  разница показана на каритнке: 

In [226]:
from keras.layers import Conv2D
model = Sequential()
# В Conv2D можно подавать параметр input_shape, если это первый слой
model.add(Conv2D(20, kernel_size=(3, 3), activation='relu', input_shape=(32, 28, 3)))

# Запись выше эквивалентна следующей записи:
model = Sequential()
model.add(InputLayer(input_shape=(32, 28, 3)))
model.add(Conv2D(1, kernel_size=(3, 3), activation='relu'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_59 (Conv2D)           (None, 30, 26, 1)         28        
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


__Вопрос:__ Нужна ли активация после Conv2D слоя? Или можно обойтись без неё?

### Reshape

Позволяет поменять размерность тензора. 

Например:

$(n, m) \rightarrow (m, n)$

$(n, m, k) \rightarrow (k, m, n)$

$(n, m, k) \rightarrow (n, m \cdot k)$

In [227]:
model = Sequential()
model.add(Dense(40, input_shape=(32, 28)))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 32, 40)            1160      
Total params: 1,160
Trainable params: 1,160
Non-trainable params: 0
_________________________________________________________________


In [228]:
from keras.layers import Reshape
# Примеры:
model.add(Reshape((8, 160)))
# теперь: model.output_shape == (None, 8, 160)

# также вы можете попросить Keras за вас посчитать оставшуюся размерность, просто подав -1
model.add(Reshape((-1, 4, 4)))
# теперт: model.output_shape == (None, 80, 4, 4)

In [229]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_75 (Dense)             (None, 32, 40)            1160      
_________________________________________________________________
reshape_19 (Reshape)         (None, 8, 160)            0         
_________________________________________________________________
reshape_20 (Reshape)         (None, 80, 4, 4)          0         
Total params: 1,160
Trainable params: 1,160
Non-trainable params: 0
_________________________________________________________________


__Вопрос:__ Когда может пригодиться Reshape слой?

### MaxPooling2D

<img src="https://developers.google.com/machine-learning/practica/image-classification/images/maxpool_animation.gif" alt="Drawing" style="width: 500px;"/>

Параметры MaxPool2D:
- ```pool_size:int or tuple/list of 2 int``` - tuple из двух int. Размерность окна, в котором выбирается максимальный элемент. На гифке выше pool_size=(2,2)
- ```strides: int or tuple/list of 2 int``` - отвечает за stride по вектрикали и горизонтали
- ```padding:str``` - принимает значение либо 'valid', либо 'same',  разница показана в блоке с Conv2D

In [230]:
# Пример:
from keras.layers import MaxPool2D
model = Sequential()
model.add(InputLayer(input_shape=(32, 28, 3)))
model.add(MaxPool2D())
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
max_pooling2d_55 (MaxPooling (None, 16, 14, 3)         0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


Слой MaxPool2D не имеет обучающих параметров

### Dropout

<img src="https://mlblr.com/images/dropout.gif" alt="Drawing" style="width: 600px;"/>

In [231]:
from keras.layers import Dropout
model.add(Dropout(rate=0.2))
# rate - вероятность, с короторой выключается каждый из нейронов при обучении 

__Вопрос:__ Добавляет ли Dropuot слой обучаемых параметров?

### Activations:

Наиболее популярные функции активации:

<img src="https://cdn-images-1.medium.com/max/1200/1*ZafDv3VUm60Eh10OeJu1vw.png" alt="Drawing" style="width: 500px;"/>

Более подробно про функции активации в Keras:
https://keras.io/activations/

In [0]:
# Два способа добавить активацию:
from keras.layers import ReLU

# Первый
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu' ,input_shape=(32, 28, 3)))

# Второй
model = Sequential()
model.add(Conv2D(32, (3, 3),input_shape=(32, 28, 3)))
model.add(ReLU())

In [0]:
# Другие возможные функции активации:
keras.layers.ReLU()
keras.layers.LeakyReLU()
keras.layers.ELU()
keras.layers.Softmax()

__Вопрос:__ В чем проблема функции sigmoid? А в чем проблема функции relu? Когда используется softmax?

## Задача:

Создать свою нейронную сеть с:
- На вход подается черно-белое изображение размером (28, 28) 
- В сеть добавить 3 блока Свертка-MaxPool. Каждый из MaxPool имеет окно (2, 2), Число фильтров для сверток: (16, 32, 64), размеры окна для сверток во всех случаях (3, 3)
- После первого блока Conv-MaxPool добавить Dropout слой с rate=0.3
- В конце сети добавить 2 слоя Dense, первый размерность выхода (batch_size, 32). Пусть число возможных классов 10, поэтому второй Dense слой имеет размерность на выходе (batch_size, 10)
- Не забудьте добавить softmax как функция активации для последнего слоя

In [0]:
# <Your Code>

In [0]:
# Проверьте правильно ли вы построили модель:
assert model.count_params() == 25706

Теперь скомпилируем модель. Задача многоклассовой классификации, какую функцию потерь будем использовать?
В качестве оптимизатора можете выбрать свой. Метрики - accuracy

In [0]:
# <Your Code>

In [0]:
# Загружаем MNIST датасет:
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train.shape

In [0]:
# Визуализация датасета:
from matplotlib import pyplot as plt
%matplotlib inline
# preview the images first
plt.figure(figsize=(12,10))
x, y = 10, 4
for i in range(40):  
    plt.subplot(y, x, i+1)
    plt.imshow(x_train[i].reshape((28,28)),interpolation='nearest')
plt.show()

In [0]:
# Здесь сделайте fit
# <Your Code>

In [0]:
# оценим итоговое качество модели на отложенной выборке: 
model.evaluate(x_test, keras.utils.to_categorical(y_test))

Ниже визуализация верных/неверных предсказаний:

In [0]:
import numpy as np
predicted_classes = model.predict_classes(x_test)

# see which we predicted correctly and which not
correct_indices = np.nonzero(predicted_classes == y_test)[0]
incorrect_indices = np.nonzero(predicted_classes != y_test)[0]
print()
print(len(correct_indices)," classified correctly")
print(len(incorrect_indices)," classified incorrectly")

# adapt figure size to accomodate 18 subplots
plt.rcParams['figure.figsize'] = (7,14)

figure_evaluation = plt.figure()

# plot 9 correct predictions
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(6,3,i+1)
    plt.imshow(x_test[correct].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
      "Predicted: {}, Truth: {}".format(predicted_classes[correct],
                                        y_test[correct]))
    plt.xticks([])
    plt.yticks([])

# plot 9 incorrect predictions
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(6,3,i+10)
    plt.imshow(x_test[incorrect].reshape(28,28), cmap='gray', interpolation='none')
    plt.title(
      "Predicted {}, Truth: {}".format(predicted_classes[incorrect], 
                                       y_test[incorrect]))
    plt.xticks([])
    plt.yticks([])

figure_evaluation

## Losses:

Различные функции потерь, которые будет оптимизировать модель
https://keras.io/losses/

In [0]:
from keras import losses

# Функция потерь передается в модель на этапе компилирования. Следующие две записи эквивалентны:
model.compile(loss='mean_squared_error', optimizer='sgd')
model.compile(loss=losses.mean_squared_error, optimizer='sgd')

# Для задачи регрессии наиболее популярные функции:
keras.losses.mean_squared_error(y_true, y_pred)
keras.losses.mean_absolute_error(y_true, y_pred)
keras.losses.mean_absolute_percentage_error(y_true, y_pred)
keras.losses.logcosh(y_true, y_pred)

# Для задачи многоклассовой классификации:
keras.losses.categorical_crossentropy(y_true, y_pred)
# Для задачи бинарной классификации:
keras.losses.binary_crossentropy(y_true, y_pred)

Или вы можете добавить кастомный лосс, который будет оптимизироваться в ходе обучения. Делать его придется при помощи backend Кераса. 

In [0]:
def customLoss(yTrue,yPred):
    return K.sum(K.log(yTrue) - K.log(yPred))

model = Sequential()
model.add(Dense(10, input_dim=100))


model.compile(loss=customLoss,
              optimizer='sgd',
              metrics=['acc'])

model.fit(x_train, y_train, epochs=3)

## Metrics:

Метрика отличается от loss function тем, что результаты метрик никак не используются в ходе обучения. Лосс - функция, которую модель оптимизирует. Метрики - показатели перфоманса модели для человека. В то же время любую из loss функций можно использовать как метрику
https://keras.io/metrics/

Метрики, как и лосс функция, подаются на жтапе компилирования модели через аргумент metrics:

In [0]:
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['mae', 'acc'])
# Другой вариант:
from keras import metrics

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=[metrics.mae, metrics.categorical_accuracy])


# Или для задачи многоклассовой классификации:
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['acc'])


Также можно сделать кастомную метрику. Делать её придется снова при помощи backend Кераса. Пример:

In [0]:
# Custom metrics:
import keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc', mean_pred])

## Optimizers:

https://keras.io/optimizers/

In [0]:
# Как обычно, можно передать через строковую переменную:
model.compile(loss='mean_squared_error', optimizer='sgd')

# а можно и через инстанс класса:
from keras import optimizers
model.compile(loss='mean_squared_error', optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False))

# Другие популярные оптимизаторы:
keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False)
keras.optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)
keras.optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0)
keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

## Visualization:

```model.fit()``` автоматически сохраняет историю обучения, которую можно визуализировать следующим образом:

In [0]:
import matplotlib.pyplot as plt

x_train = np.random.random((1000, 100))
y_train = np_utils.to_categorical(np.random.randint(10, size=(1000, 1)))

model = Sequential()
model.add(InputLayer(input_shape=(100, )))
#model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['acc'])

history = model.fit(x_train, y_train, validation_split=0.25, epochs=50, batch_size=16, verbose=1)

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Image Preprocessing

Для аугментации изображений можно использовать ImageDataGenerator

In [0]:
# Объявляем объект из класса ImageDataGenerator.
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# фитим его на данные
datagen.fit(x_train)

# делаем fit модели с аугментацией данных
model.fit_generator(datagen.flow(x_train, y_train, batch_size=16),
                    steps_per_epoch=len(x_train) / 32, epochs=5)

# или можно то же самое в цикле:
for e in range(5):
    print('Epoch', e)
    batches = 0
    for x_batch, y_batch in datagen.flow(x_train, y_train, batch_size=16):
        model.fit(x_batch, y_batch)
        batches += 1
        if batches >= len(x_train) / 32:
            break

## Некоторые вопросы из FAQ:

1.Как можно импортировать обученные модели в Keras?

In [0]:
from keras.applications.xception import Xception
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.applications.resnet50 import ResNet50
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.mobilenet import MobileNet
from keras.applications.densenet import DenseNet121
from keras.applications.densenet import DenseNet169
from keras.applications.densenet import DenseNet201
from keras.applications.nasnet import NASNetLarge
from keras.applications.nasnet import NASNetMobile
from keras.applications.mobilenet_v2 import MobileNetV2

model = VGG16(weights='imagenet', include_top=True)

 2.Как заморозить обучение некоторых слоев в нейронной сети?

In [0]:
# у каждого слоя есть параметр trainable, по умолчанию он true
for layer in model.layers[:1]:
    layer.trainable = False

3.Как мне удалить слой из модели Sequential?

In [0]:
model = Sequential()
model.add(Dense(32, activation='relu', input_dim=784))
model.add(Dense(32, activation='relu'))

print(len(model.layers))  # "2"

model.pop()
print(len(model.layers))  # "1"

In [0]:
model.summary()


In [0]:
model.input_shape

4.Как мне получить output некоторого слоя нейронной сети?

In [0]:
from keras.models import Model

#model = ...  # объявите модель

# random data:
data = np.random.randint(low=1, high=10, size=(100, model.input_shape[1]))

layer_name = model.layers[0].name
intermediate_layer_model = Model(inputs=model.input,
                                 outputs=model.get_layer(layer_name).output)
intermediate_output = intermediate_layer_model.predict(data)

5.Как мне обучать модель, если данные не помещаются в оперативную память?


Можно использовать метод model.train_on_batch(x, y), итерируясь по батчам. Метод model.test_on_batch(x, y) позволит итерируясь по тестовым данным получить оценку качества модели


Либо можно использовать метод fit_generator(), в который передается генератор данных:

In [0]:
model.fit_generator(data_generator, steps_per_epoch, epochs)

6.Как сохранять модель из Кераса?

In [0]:
model.save(filepath) # architecture, weights, loss, the state of the optimizer, allowing to resume training exactly where you left off.
keras.models.load_model(filepath)

7.Как мне запустить обучение модели на GPU? Ответ: Keras это сделает за вас автоматически
Чтобы узнать список используемых GPU и првоерить видит ли их Keras:

In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

8.Как запустить обучение модели на нескольких GPU?

In [0]:
from keras.utils import multi_gpu_model

# Replicates `model` on 8 GPUs.
# This assumes that your machine has 8 available GPUs.
parallel_model = multi_gpu_model(model, gpus=8)
parallel_model.compile(loss='categorical_crossentropy',
                       optimizer='rmsprop')

# This `fit` call will be distributed on 8 GPUs.
# Since the batch size is 256, each GPU will process 32 samples.
parallel_model.fit(x, y, epochs=20, batch_size=256)